In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
from tqdm import tqdm
from pathlib import Path
from skimage import measure
import scipy.ndimage

from postprocessingfunction import refine_lable
from loss import DiceLoss, dice_coef_metric
from model import Model

VALIDATION_FOLDER = "validationcombinebothstages"
Path(f"./{VALIDATION_FOLDER}").mkdir(parents=True, exist_ok=True)

def get_bbox(img):
    r = np.any(img, axis=(1, 2))
    c = np.any(img, axis=(0, 2))
    z = np.any(img, axis=(0, 1))

    rmin, rmax = np.where(r)[0][[0, -1]]
    cmin, cmax = np.where(c)[0][[0, -1]]
    zmin, zmax = np.where(z)[0][[0, -1]]

    return rmin, rmax, cmin, cmax, zmin, zmax


def get_padded_bbox(labels, padding=0.1):
    xmin0, xmax0, ymin0, ymax0, zmin0, zmax0 = get_bbox(labels[0])
    xmin1, xmax1, ymin1, ymax1, zmin1, zmax1 = get_bbox(labels[1])
    xmin2, xmax2, ymin2, ymax2, zmin2, zmax2 = get_bbox(labels[2])
    
    xmin = min(xmin0, xmin1, xmin2)
    ymin = min(ymin0, ymin1, ymin2)
    zmin = min(zmin0, zmin1, zmin2)
    xmax = max(xmax0, xmax1, xmax2)
    ymax = max(ymax0, ymax1, ymax2)
    zmax = max(zmax0, zmax1, zmax2)

    cropped = labels[xmin:xmax, ymin:ymax, zmin:zmax]

    # make cuboid
    longest_axis = np.argmax(cropped.shape)
    length = max(cropped.shape)
    bbox = [xmin, ymin, zmin, xmax, ymax, zmax]
    cubebox = []
    for i in range(3):
        start = bbox[i]
        end = bbox[i+3]
        padstart = round((length - (end - start)) / 2)
        padend = (length - (end - start)) // 2
        cubebox.append(start - padstart)
        cubebox.append(end + padend)
    xmin, xmax, ymin, ymax, zmin, zmax = cubebox

    # add padding
    xmin -= round(padding * length)
    ymin -= round(padding * length)
    zmin -= round(padding * length)
    xmax += round(padding * length)
    ymax += round(padding * length)
    zmax += round(padding * length)

    # make sure coords are within bounds
    xmin = max(0, xmin)
    ymin = max(0, ymin)
    zmin = max(0, zmin)
    xmax = min(240, xmax)
    ymax = min(240, ymax)
    zmax = min(155, zmax)

    return xmin, xmax, ymin, ymax, zmin, zmax

def pad_to_cuboid(arr):
    xlen = arr.shape[0]
    ylen = arr.shape[1]
    zlen = arr.shape[2]
    if (xlen != ylen) or (ylen != zlen) or (zlen != xlen):
        size = max(xlen, ylen, zlen)

        a = (size - xlen) // 2
        aa = size - a - xlen

        b = (size - ylen) // 2
        bb = size - b - ylen

        c = (size - zlen) // 2
        cc = size - c - zlen

        return np.pad(arr, [(a, aa), (b, bb), (c, cc)], mode='constant')
    return arr
    
def pad_x_to_cuboid(arr):
    xlen = arr.shape[1]
    ylen = arr.shape[2]
    zlen = arr.shape[3]
    if (xlen != ylen) or (ylen != zlen) or (zlen != xlen):
        size = max(xlen, ylen, zlen)

        a = (size - xlen) // 2
        aa = size - a - xlen

        b = (size - ylen) // 2
        bb = size - b - ylen

        c = (size - zlen) // 2
        cc = size - c - zlen

        return np.pad(arr, [(0, 0), (a, aa), (b, bb), (c, cc)], mode='constant')
    return arr

def preprocess_mask_labels(mask: np.ndarray):
    mask_WT = mask.copy()
    mask_WT[mask_WT == 1] = 1
    mask_WT[mask_WT == 2] = 1
    mask_WT[mask_WT == 4] = 1

    mask_TC = mask.copy()
    mask_TC[mask_TC == 1] = 1
    mask_TC[mask_TC == 2] = 0
    mask_TC[mask_TC == 4] = 1

    mask_ET = mask.copy()
    mask_ET[mask_ET == 1] = 0
    mask_ET[mask_ET == 2] = 0
    mask_ET[mask_ET == 4] = 1

    mask = np.stack([mask_TC, mask_WT , mask_ET])
    return mask
    
def normalize(data: np.ndarray):
    return (data - data.mean()) / data.std()

def dice(prob, truth, eps: float = 1e-9):
    pred = (prob >= 0.5)
    intersection = 2.0 * (pred * truth).sum()
    union = truth.sum() + pred.sum()
    if truth.sum() == 0 and pred.sum() == 0:
        return 1.0
    else:
        return ((intersection + eps) / union).item()

def dice_overall(probabilities, truth, threshold: float = 0.5, eps: float = 1e-9):
    scores = []
    num = probabilities.shape[0]
    predictions = (probabilities >= threshold)
    assert(predictions.shape == truth.shape)
    for i in range(num):
        prediction = predictions[i]
        truth_ = truth[i]
        intersection = 2.0 * (truth_ * prediction).sum()
        union = truth_.sum() + prediction.sum()
        if truth_.sum() == 0 and prediction.sum() == 0:
            scores.append(1.0)
        else:
            scores.append(((intersection + eps) / union).item())
    return np.mean(scores)


PATH = "./bbox_models/d3c40newdata.pt"
bbox_model = Model(in_channels=4, n_classes=3, n_channels=40)
bbox_model.load_state_dict(torch.load(PATH))
bbox_model.eval()
print(f"loaded {PATH}")

PATH = "./segment_models/d3c40bbox.pt"
segment_model = Model(in_channels=4, n_classes=3, n_channels=40)
segment_model.load_state_dict(torch.load(PATH))
segment_model.eval()
print(f"loaded {PATH}")

rootPath = "/projects/ab57/data/BraTS2021_Validation_Combined/"

subject = []

# sizes
NULLs = []
ETs = []
WTs = []
TCs = []

print(f"using path {rootPath}")
with os.scandir(rootPath) as it:
    for entry in tqdm(it):
        ''' First Stage '''
        x = np.load(entry.path)

        # scale down image
        # x = scipy.ndimage.zoom(x, (1, 0.5, 0.5, 0.5), order=0)

        # crop to brain extent
        rmin, rmax, cmin, cmax, zmin, zmax = get_bbox(x[0])
        x = x[:, rmin:rmax, cmin:cmax, zmin:zmax]

        x = normalize(x)
        x = torch.from_numpy(x)
        x = x.unsqueeze(0)
        x = x.float()

        pred = bbox_model(x)
        pred = torch.sigmoid(pred)[0]
        pred1 = pred.detach().cpu().numpy()

        # scale up prediction
        # pred = scipy.ndimage.zoom(pred, (1, 2, 2, 2), order=0)

        ''' Second Stage '''
        x = np.load(entry.path)

        # crop to brain extent
        rmin, rmax, cmin, cmax, zmin, zmax = get_bbox(x[0])
        x = x[:, rmin:rmax, cmin:cmax, zmin:zmax]

        # crop to tumor bounding box
        rmint, rmaxt, cmint, cmaxt, zmint, zmaxt = get_padded_bbox(pred1)
        x = x[:, rmint:rmaxt, cmint:cmaxt, zmint:zmaxt]

        # x = pad_x_to_cuboid(x)
        x = normalize(x)
        x = torch.from_numpy(x)
        x = x.unsqueeze(0)
        x = x.float()

        pred = segment_model(x)
        pred = torch.sigmoid(pred)[0]
        pred2 = pred.detach().cpu().numpy()

        ''' Combine Predictions '''
        # reapply padding to both outputs
        pred1 = np.pad(pred1, ((0, 0), (rmin, 240-rmin-pred1.shape[1]), (cmin, 240-cmin-pred1.shape[2]), (zmin, 155-zmin-pred1.shape[3])), 'constant', constant_values=(0))
        pred2 = np.pad(pred2, ((0, 0), (rmin+rmint, 240-(rmin+rmint)-pred2.shape[1]), (cmin+cmint, 240-(cmin+cmint)-pred2.shape[2]), (zmin+zmint, 155-(zmin+zmint)-pred2.shape[3])), 'constant', constant_values=(0))

        # take average of the two
        pred = np.mean(np.array([ pred1, pred2 ]), axis=0)
        pred = np.rint(pred)

        ''' Create Submission Files '''
        func = nib.load("/projects/ab57/data/RSNA_ASNR_MICCAI_BraTS2021_ValidationData/BraTS2021_00082/BraTS2021_00082_t1.nii.gz")

        print("reshaping prediction...")
        # 1 for TC, 2 for WT, 4 for ET,
        TC = pred[0]
        WT = pred[1] * 2
        
        if np.count_nonzero(pred[2]) < 500:
            print("ET being converted to TC")
            ET = pred[2]
        else:
            ET = pred[2] * 4
        
        y = np.where(ET != 0, ET, TC)
        y = np.where(y != 0, y, WT)
        print(y.shape)
        print("saving prediction to nifti file...")
        ni_img = nib.Nifti1Image(y, func.affine)
        nib.save(ni_img, f"./{VALIDATION_FOLDER}/{entry.name[:-4]}.nii.gz")

        ''' Calculate Prediction Tumour Sizes '''
        subregions, sizes = np.unique(y, return_counts=True)
        if 1 not in subregions:
            sizes = np.insert(sizes, 1, 0)
        if 2 not in subregions:
            sizes = np.insert(sizes, 2, 0)
        if 4 not in subregions:
            sizes = np.insert(sizes, 3, 0)
        
        NULL_area, TC_area, WT_area, ET_area = sizes
        NULLs.append(NULL_area)
        ETs.append(ET_area)
        WTs.append(WT_area)
        TCs.append(TC_area)
        subject.append(entry.path.split('/')[-1])

print('writing sizes to file')
import csv
rows = zip(subject, WTs, TCs, ETs, NULLs)
with open("./combinebothstagesexternal.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(['SUBJECT', 'WT_area', 'TC_area', 'ET_area', 'NULL_area'])
    for row in rows:
        writer.writerow(row)

loaded ./bbox_models/d3c40newdata.pt


0it [00:00, ?it/s]

loaded ./segment_models/d3c40bbox.pt
using path /projects/ab57/data/BraTS2021_Validation_Combined/
reshaping prediction...
(240, 240, 155)
saving prediction to nifti file...


1it [00:11, 11.62s/it]

reshaping prediction...
(240, 240, 155)
saving prediction to nifti file...


...

reshaping prediction...
ET being converted to TC
(240, 240, 155)
saving prediction to nifti file...


218it [43:13, 11.65s/it]

reshaping prediction...
(240, 240, 155)
saving prediction to nifti file...


219it [43:27, 11.91s/it]

writing sizes to file
